## Extracting Data from S3

In [1]:
import pandas as pd
import boto3
from io import StringIO
import configparser
from sqlalchemy import create_engine
import psycopg2
from urllib.parse import quote_plus

In [2]:
config = configparser.ConfigParser()
config.read('/home/sakshi/spotify_project/config.ini')

['/home/sakshi/spotify_project/config.ini']

In [3]:
aws_access_key_id = config['aws']['aws_access_key_id']
aws_secret_access_key = config['aws']['aws_secret_access_key']
bucket_name = config['aws']['s3_bucket']
tracks_folder = config['aws']['s3_tracks_folder']
artists_folder = config['aws']['s3_artists_folder']

In [4]:
s3 = boto3.client(
    's3',
    aws_access_key_id=aws_access_key_id,
    aws_secret_access_key=aws_secret_access_key
)

In [5]:
def merge_csvs_from_s3(bucket_name, folder_name):
    response = s3.list_objects_v2(Bucket=bucket_name, Prefix=folder_name)
    
    if 'Contents' not in response:
        print(f"No files found in {folder_name}")
        return pd.DataFrame() 
    
    files = response['Contents']
    
    merged_df = pd.DataFrame()
    
    for file in files:
        key = file['Key']
        if key.endswith('.csv'):
            file_obj = s3.get_object(Bucket=bucket_name, Key=key)
            file_content = file_obj['Body'].read().decode('utf-8')
            

            temp_df = pd.read_csv(StringIO(file_content))
            merged_df = pd.concat([merged_df, temp_df], ignore_index=True)
    
    return merged_df

tracks_df = merge_csvs_from_s3(bucket_name, tracks_folder)
artists_df = merge_csvs_from_s3(bucket_name, artists_folder)

In [6]:
tracks_df.head()

,track_id,track_name,album_artist_id,album_artist_name,duration_ms,explicit,popularity,track_number,played_at,track_url,artist_url,album_id,album_name,album_release_date,total_tracks,album_url,context_type,playlist_url
0,3zSSCPpLZ5Oc8nelhhGjKz,Tu hai kahan,2cjQTf2J5yCaNY8qHpW855,AUR,263184,False,75,1,2024-10-13 10:27:15.217,https://open.spotify.com/track/3zSSCPpLZ5Oc8ne...,https://open.spotify.com/artist/2cjQTf2J5yCaNY...,73vfz6Szer7i3GFIesix6l,Tu hai kahan,2023-10-16,1,https://open.spotify.com/album/73vfz6Szer7i3GF...,NaN,NaN
1,6N4TjPeZi4S9jpayOpRX2h,Yaarr Ni Milyaa,4ITkqBlf5eoVCOFwsJCnqo,Harrdy Sandhu,310208,False,61,1,2024-10-13 10:11:59.811,https://open.spotify.com/track/6N4TjPeZi4S9jpa...,https://open.spotify.com/artist/4ITkqBlf5eoVCO...,40YiNI76fpTK5IXRv4VbHP,Yaarr Ni Milyaa,2017-09-05,1,https://open.spotify.com/album/40YiNI76fpTK5IX...,NaN,NaN
2,3KkXRkHbMCARz0aVfEt68P,Sunflower - Spider-Man: Into the Spider-Verse,246dkjvS1zLTtiykXe5h60,Post Malone,158040,False,80,2,2024-10-13 10:06:49.125,https://open.spotify.com/track/3KkXRkHbMCARz0a...,https://open.spotify.com/artist/246dkjvS1zLTti...,35s58BRTGAEWztPo9WqCIs,Spider-Man: Into the Spider-Verse (Soundtrack ...,2018-12-14,13,https://open.spotify.com/album/35s58BRTGAEWztP...,NaN,NaN
3,1LNlfvPQmB0cqYJQQskZ8x,MOMMAE (Feat. Ugly Duck),4XDi67ZENZcbfKnvMnTYsI,Jay Park,204991,True,65,4,2024-10-13 09:59:01.287,https://open.spotify.com/track/1LNlfvPQmB0cqYJ...,https://open.spotify.com/artist/4XDi67ZENZcbfK...,5vESroqrGYDxDPAwUceQxf,WORLDWIDE,2015-11-05,18,https://open.spotify.com/album/5vESroqrGYDxDPA...,NaN,NaN
4,2gkVEnpahpE3bQuvGuCpAV,Yes or No,6HaGTQPmzraVmaVxvz6EUc,Jung Kook,147557,False,76,5,2024-10-12 09:01:38.462,https://open.spotify.com/track/2gkVEnpahpE3bQu...,https://open.spotify.com/artist/6HaGTQPmzraVma...,5pSk3c3wVwnb2arb6ohCPU,GOLDEN,2023-11-03,11,https://open.spotify.com/album/5pSk3c3wVwnb2ar...,playlist,https://open.spotify.com/playlist/37i9dQZF1EQp...


In [7]:
tracks_df.count()

track_id              2651
track_name            2651
album_artist_id       2651
album_artist_name     2651
duration_ms           2651
explicit              2651
popularity            2651
track_number          2651
played_at             2651
track_url             2651
artist_url            2651
album_id              2651
album_name            2651
album_release_date    2651
total_tracks          2651
album_url             2651
context_type           102
playlist_url           102
dtype: int64

In [8]:
artists_df.head()

,track_id,artist_id,artist_name
0,2plbrEY59IikOBgBGLjaoe,1HY2Jd0NmPuamShAr6KMms,Lady Gaga
1,2plbrEY59IikOBgBGLjaoe,0du5cEVh5yTK9QJze8zA0C,Bruno Mars
2,2CspwnypzT7rcWI9RfsoSb,250b0Wlc5Vk0CoUsaCY84M,JENNIE
3,5jsw9uXEGuKyJzs0boZ1bT,7n2wHs1TKAczGzO7Dd2rGr,Shawn Mendes
4,01prGWjCTxKBxJc400zwvQ,5L1lO4eRHmJ7a0Q6csE5cT,LISA


In [9]:
artists_df.count()

track_id       4392
artist_id      4392
artist_name    4392
dtype: int64

## Transformation 

In [10]:
new_tracks = tracks_df.drop_duplicates()

In [11]:
new_tracks.count()

track_id              1113
track_name            1113
album_artist_id       1113
album_artist_name     1113
duration_ms           1113
explicit              1113
popularity            1113
track_number          1113
played_at             1113
track_url             1113
artist_url            1113
album_id              1113
album_name            1113
album_release_date    1113
total_tracks          1113
album_url             1113
context_type            66
playlist_url            66
dtype: int64

In [12]:
new_artists = artists_df.drop_duplicates()

In [13]:
new_artists.count()

track_id       1200
artist_id      1200
artist_name    1200
dtype: int64

In [14]:
new_tracks.dtypes

track_id              object
track_name            object
album_artist_id       object
album_artist_name     object
duration_ms            int64
explicit                bool
popularity             int64
track_number           int64
played_at             object
track_url             object
artist_url            object
album_id              object
album_name            object
album_release_date    object
total_tracks           int64
album_url             object
context_type          object
playlist_url          object
dtype: object

In [15]:
new_tracks = new_tracks.copy()
new_tracks['played_at_date'] = new_tracks['played_at']

In [16]:
new_tracks[['track_name','played_at_date']]

,track_name,played_at_date
0,Tu hai kahan,2024-10-13 10:27:15.217
1,Yaarr Ni Milyaa,2024-10-13 10:11:59.811
2,Sunflower - Spider-Man: Into the Spider-Verse,2024-10-13 10:06:49.125
3,MOMMAE (Feat. Ugly Duck),2024-10-13 09:59:01.287
4,Yes or No,2024-10-12 09:01:38.462
...,...,...
2596,Fighting (Feat. Lee Young Ji),2024-11-30 12:53:52.566000+05:30
2597,Fat Funny Friend,2024-11-30 12:50:27.566000+05:30
2598,Happier Than Ever - Edit,2024-11-30 12:47:07.073000+05:30
2599,Sparkle - movie ver.,2024-11-30 12:43:43.540000+05:30


In [17]:
new_tracks['played_at_date'] = new_tracks['played_at_date'].str.split(' ').str[0]

In [18]:
new_tracks['played_at_date'] = pd.to_datetime(new_tracks['played_at_date'])

In [19]:
new_tracks['played_at'] = new_tracks['played_at'].str.replace(r'\+.*', '', regex=True)

In [20]:
new_tracks['played_at'] = new_tracks['played_at'].str.split('.').str[0]

In [21]:
new_tracks['played_at'] = pd.to_datetime(new_tracks['played_at']).dt.strftime('%Y-%m-%d %H:%M:%S')

In [22]:
new_tracks

,track_id,track_name,album_artist_id,album_artist_name,duration_ms,explicit,popularity,track_number,played_at,track_url,artist_url,album_id,album_name,album_release_date,total_tracks,album_url,context_type,playlist_url,played_at_date
0,3zSSCPpLZ5Oc8nelhhGjKz,Tu hai kahan,2cjQTf2J5yCaNY8qHpW855,AUR,263184,False,75,1,2024-10-13 10:27:15,https://open.spotify.com/track/3zSSCPpLZ5Oc8ne...,https://open.spotify.com/artist/2cjQTf2J5yCaNY...,73vfz6Szer7i3GFIesix6l,Tu hai kahan,2023-10-16,1,https://open.spotify.com/album/73vfz6Szer7i3GF...,NaN,NaN,2024-10-13
1,6N4TjPeZi4S9jpayOpRX2h,Yaarr Ni Milyaa,4ITkqBlf5eoVCOFwsJCnqo,Harrdy Sandhu,310208,False,61,1,2024-10-13 10:11:59,https://open.spotify.com/track/6N4TjPeZi4S9jpa...,https://open.spotify.com/artist/4ITkqBlf5eoVCO...,40YiNI76fpTK5IXRv4VbHP,Yaarr Ni Milyaa,2017-09-05,1,https://open.spotify.com/album/40YiNI76fpTK5IX...,NaN,NaN,2024-10-13
2,3KkXRkHbMCARz0aVfEt68P,Sunflower - Spider-Man: Into the Spider-Verse,246dkjvS1zLTtiykXe5h60,Post Malone,158040,False,80,2,2024-10-13 10:06:49,https://open.spotify.com/track/3KkXRkHbMCARz0a...,https://open.spotify.com/artist/246dkjvS1zLTti...,35s58BRTGAEWztPo9WqCIs,Spider-Man: Into the Spider-Verse (Soundtrack ...,2018-12-14,13,https://open.spotify.com/album/35s58BRTGAEWztP...,NaN,NaN,2024-10-13
3,1LNlfvPQmB0cqYJQQskZ8x,MOMMAE (Feat. Ugly Duck),4XDi67ZENZcbfKnvMnTYsI,Jay Park,204991,True,65,4,2024-10-13 09:59:01,https://open.spotify.com/track/1LNlfvPQmB0cqYJ...,https://open.spotify.com/artist/4XDi67ZENZcbfK...,5vESroqrGYDxDPAwUceQxf,WORLDWIDE,2015-11-05,18,https://open.spotify.com/album/5vESroqrGYDxDPA...,NaN,NaN,2024-10-13
4,2gkVEnpahpE3bQuvGuCpAV,Yes or No,6HaGTQPmzraVmaVxvz6EUc,Jung Kook,147557,False,76,5,2024-10-12 09:01:38,https://open.spotify.com/track/2gkVEnpahpE3bQu...,https://open.spotify.com/artist/6HaGTQPmzraVma...,5pSk3c3wVwnb2arb6ohCPU,GOLDEN,2023-11-03,11,https://open.spotify.com/album/5pSk3c3wVwnb2ar...,playlist,https://open.spotify.com/playlist/37i9dQZF1EQp...,2024-10-12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2596,7eBpUuPnDTfbeP1P4P93CS,Fighting (Feat. Lee Young Ji),1uAT5bTSp6dWbNmixIUP5t,BSS,204466,False,67,1,2024-11-30 12:53:52,https://open.spotify.com/track/7eBpUuPnDTfbeP1...,https://open.spotify.com/artist/1uAT5bTSp6dWbN...,4dHtpne5cAAGgDYFNHu7jW,BSS 1st Single Album 'SECOND WIND',2023-02-06,3,https://open.spotify.com/album/4dHtpne5cAAGgDY...,NaN,NaN,2024-11-30
2597,7z3NjKBqjv3d1ykXCFQxaT,Fat Funny Friend,5dtPlx7yKOo7KdZGyrfFIq,Maddie Zahm,199764,False,64,1,2024-11-30 12:50:27,https://open.spotify.com/track/7z3NjKBqjv3d1yk...,https://open.spotify.com/artist/5dtPlx7yKOo7Kd...,5NMKRqxbne8L5LerDnpGjk,Fat Funny Friend,2022-02-04,1,https://open.spotify.com/album/5NMKRqxbne8L5Le...,NaN,NaN,2024-11-30
2598,32BeYxKPrig1LefHsC0Xuo,Happier Than Ever - Edit,6qqNVTkY8uBg9cP3Jd7DAH,Billie Eilish,151444,True,69,1,2024-11-30 12:47:07,https://open.spotify.com/track/32BeYxKPrig1Lef...,https://open.spotify.com/artist/6qqNVTkY8uBg9c...,2kzPJWrTjVKEYWWhowXLnz,Happier Than Ever (Edit),2021-07-28,1,https://open.spotify.com/album/2kzPJWrTjVKEYWW...,NaN,NaN,2024-11-30
2599,3A4FRzgve9BjfKbvVXRIFO,Sparkle - movie ver.,1EowJ1WwkMzkCkRomFhui7,RADWIMPS,537653,False,64,24,2024-11-30 12:43:43,https://open.spotify.com/track/3A4FRzgve9BjfKb...,https://open.spotify.com/artist/1EowJ1WwkMzkCk...,4qApTp9557qYZzRLEih4uP,Your Name.,2016-08-24,27,https://open.spotify.com/album/4qApTp9557qYZzR...,NaN,NaN,2024-11-30


In [23]:
new_tracks[['track_name','explicit']].where(new_tracks['explicit'] == True).count()

track_name    88
explicit      88
dtype: int64

In [24]:
new_artists.dtypes

track_id       object
artist_id      object
artist_name    object
dtype: object

## Dimension Modelling 

In [25]:
new_tracks.head()

,track_id,track_name,album_artist_id,album_artist_name,duration_ms,explicit,popularity,track_number,played_at,track_url,artist_url,album_id,album_name,album_release_date,total_tracks,album_url,context_type,playlist_url,played_at_date
0,3zSSCPpLZ5Oc8nelhhGjKz,Tu hai kahan,2cjQTf2J5yCaNY8qHpW855,AUR,263184,False,75,1,2024-10-13 10:27:15,https://open.spotify.com/track/3zSSCPpLZ5Oc8ne...,https://open.spotify.com/artist/2cjQTf2J5yCaNY...,73vfz6Szer7i3GFIesix6l,Tu hai kahan,2023-10-16,1,https://open.spotify.com/album/73vfz6Szer7i3GF...,NaN,NaN,2024-10-13
1,6N4TjPeZi4S9jpayOpRX2h,Yaarr Ni Milyaa,4ITkqBlf5eoVCOFwsJCnqo,Harrdy Sandhu,310208,False,61,1,2024-10-13 10:11:59,https://open.spotify.com/track/6N4TjPeZi4S9jpa...,https://open.spotify.com/artist/4ITkqBlf5eoVCO...,40YiNI76fpTK5IXRv4VbHP,Yaarr Ni Milyaa,2017-09-05,1,https://open.spotify.com/album/40YiNI76fpTK5IX...,NaN,NaN,2024-10-13
2,3KkXRkHbMCARz0aVfEt68P,Sunflower - Spider-Man: Into the Spider-Verse,246dkjvS1zLTtiykXe5h60,Post Malone,158040,False,80,2,2024-10-13 10:06:49,https://open.spotify.com/track/3KkXRkHbMCARz0a...,https://open.spotify.com/artist/246dkjvS1zLTti...,35s58BRTGAEWztPo9WqCIs,Spider-Man: Into the Spider-Verse (Soundtrack ...,2018-12-14,13,https://open.spotify.com/album/35s58BRTGAEWztP...,NaN,NaN,2024-10-13
3,1LNlfvPQmB0cqYJQQskZ8x,MOMMAE (Feat. Ugly Duck),4XDi67ZENZcbfKnvMnTYsI,Jay Park,204991,True,65,4,2024-10-13 09:59:01,https://open.spotify.com/track/1LNlfvPQmB0cqYJ...,https://open.spotify.com/artist/4XDi67ZENZcbfK...,5vESroqrGYDxDPAwUceQxf,WORLDWIDE,2015-11-05,18,https://open.spotify.com/album/5vESroqrGYDxDPA...,NaN,NaN,2024-10-13
4,2gkVEnpahpE3bQuvGuCpAV,Yes or No,6HaGTQPmzraVmaVxvz6EUc,Jung Kook,147557,False,76,5,2024-10-12 09:01:38,https://open.spotify.com/track/2gkVEnpahpE3bQu...,https://open.spotify.com/artist/6HaGTQPmzraVma...,5pSk3c3wVwnb2arb6ohCPU,GOLDEN,2023-11-03,11,https://open.spotify.com/album/5pSk3c3wVwnb2ar...,playlist,https://open.spotify.com/playlist/37i9dQZF1EQp...,2024-10-12


In [26]:
new_artists.head()

,track_id,artist_id,artist_name
0,2plbrEY59IikOBgBGLjaoe,1HY2Jd0NmPuamShAr6KMms,Lady Gaga
1,2plbrEY59IikOBgBGLjaoe,0du5cEVh5yTK9QJze8zA0C,Bruno Mars
2,2CspwnypzT7rcWI9RfsoSb,250b0Wlc5Vk0CoUsaCY84M,JENNIE
3,5jsw9uXEGuKyJzs0boZ1bT,7n2wHs1TKAczGzO7Dd2rGr,Shawn Mendes
4,01prGWjCTxKBxJc400zwvQ,5L1lO4eRHmJ7a0Q6csE5cT,LISA


## Jupyter to Postgres

In [27]:
config = configparser.ConfigParser()
config.read('config.ini')

In [ ]:
dbname = config['database']['dbname']
user = config['database']['user']
password = config['database']['password']
port = int(config['database']['port'])
host = config['database']['host']

In [49]:
try:
    conn = psycopg2.connect(dbname=dbname, user=user, password=password, host=host, port=port)
    print("Database connection successful with psycopg2")
except psycopg2.Error as e:
    print(f"Database connection error with psycopg2: {e}")

In [50]:
cur = conn.cursor()

In [51]:
quoted_password = quote_plus(password)

In [52]:
engine = create_engine(f'postgresql+psycopg2://{user}:{quoted_password}@{host}:{port}/{dbname}')

In [53]:
try:
    cur.execute("""
    DROP TABLE IF EXISTS tracks;
    CREATE TABLE tracks(
    track_id varchar,
    track_name varchar,
    album_artist_id varchar,
    album_artist_name varchar,
    duration_ms int,
    explicit bool,
    popularity int,
    track_number int,
    played_at timestamp,
    track_url text,
    artist_url text,
    album_id varchar,
    album_name varchar,
    album_release_date date,
    total_tracks int,
    album_url varchar,
    context_type varchar,
    playlist_url text,
    played_at_date date
    );
    """)
    conn.commit()
    print('Table Created Successfully')
except psycopg2.Error as e:
    print(e)
    conn.rollback()

Table Created Successfully


In [54]:
new_tracks.to_sql('tracks',con=engine,if_exists='replace',index=False)

113

In [56]:
try:
    cur.execute("""
    DROP TABLE IF EXISTS artists;
    CREATE TABLE artists(
    track_id varchar,
    artist_id varchar,
    artist_name varchar
    );
    """)
    conn.commit()
    print('Table Created Successfully')
except psycopg2.Error as e:
    print(e)
finally:
    cur.close()
    conn.close()


Table Created Successfully


In [57]:
new_artists.to_sql('artists',con=engine,if_exists='replace',index=False)

200